In [1]:
import random
import matplotlib.pyplot as plt
import numpy as np
import cobra
from cobra.io import read_sbml_model
from cobra.medium import minimal_medium
from cobra import Model, Reaction, Metabolite
from cobra.flux_analysis import flux_variability_analysis
from os import listdir
from os.path import isfile, join
import pickle
import scipy.cluster.hierarchy as sch
from scipy import stats

In [2]:
#model = read_sbml_model('/Users/magdalena/polybox/research/projecto_Sebastian/Emily_Yamini/D2M19_gapfilled_denitrification_Nov23.xml')
model = read_sbml_model('D2M19.xml')

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-17


In [3]:
print('%i reaction' % len(model.reactions))
print('%i metabolites' % len(model.metabolites))
print('%i genes' % len(model.genes))

2074 reaction
1392 metabolites
1032 genes


In [4]:
media_models = ['EX_h2o_e',
'EX_o2_e',
'EX_cl_e',
'EX_na1_e',
'EX_so4_e',
'EX_k_e',
'EX_mg2_e',
'EX_ca2_e',
'EX_nh4_e',
'EX_pi_e',
'EX_btn_e',
'EX_fol_e',
'EX_pydxn_e',
'EX_ribflv_e',
'EX_thm_e',
'EX_nac_e',
'EX_ala_B_e',
'EX_4abz_e',
'EX_fe3_e',
'EX_h_e',
'EX_cobalt2_e',
'EX_cu2_e',
'EX_mn2_e',
'EX_mobd_e',
'EX_zn2_e',
'EX_sel_e',
'EX_ni2_e',
'EX_co2_e',
'EX_tungs_e',
'EX_ascb__L_e',
'EX_nad_e',
'DM_thmpp_c',
'EX_cbl1_e']#medim mimiking composition of MBL medium from Sammy (see data_curated_draft.xls in MaryAnn folder)

In [5]:
for ex in model.exchanges:
    if ex.id in media_models:
        ex.lower_bound = -1000
    else:
        ex.lower_bound = 0
        
sol = model.optimize() #no growth without carbon source
print('Growth without carbon source: ', sol.objective_value)

#carbon source
model.reactions.get_by_id('EX_lac__D_e').lower_bound = -10

#without o2 and with no3 
model.reactions.get_by_id('EX_o2_e').lower_bound = 0
model.reactions.get_by_id('EX_no3_e').lower_bound = -1000

#without no secretion (as in Xin's model)
model.reactions.get_by_id('EX_no_e').upper_bound = 0

sol = model.optimize()
print(model.summary())


Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.


Growth without carbon source:  0.0
Objective
1.0 Growth = 0.3936086137748

Uptake
------
Metabolite     Reaction      Flux  C-Number C-Flux
     ca2_e     EX_ca2_e  0.001984         0  0.00%
      cl_e      EX_cl_e  0.001984         0  0.00%
 cobalt2_e EX_cobalt2_e 3.812E-05         0  0.00%
     cu2_e     EX_cu2_e 0.0002703         0  0.00%
     fe3_e     EX_fe3_e  0.005536         0  0.00%
       h_e       EX_h_e     7.336         0  0.00%
       k_e       EX_k_e    0.0744         0  0.00%
  lac__D_e  EX_lac__D_e        10         0  0.00%
     mg2_e     EX_mg2_e  0.003307         0  0.00%
     mn2_e     EX_mn2_e 0.0002634         0  0.00%
     nac_e     EX_nac_e 0.0008683         0  0.00%
     nh4_e     EX_nh4_e     4.267         0  0.00%
     no3_e     EX_no3_e     26.47         0  0.00%
      pi_e      EX_pi_e    0.3759         0  0.00%
     so4_e     EX_so4_e   0.09545         0  0.00%
     thm_e     EX_thm_e   8.5E-05         0  0.00%
     zn2_e     EX_zn2_e   0.00013         0 

In [6]:
##
#change the N product secreted - 1, 2 or 3 steps

#1-step pathway
print('1-step pathway: NO3 -> NO2')
model.reactions.get_by_id('EX_n2o_e').upper_bound = 0
model.reactions.get_by_id('EX_n2_e').upper_bound = 0

sol = model.optimize()
print(model.summary())

model.reactions.get_by_id('EX_n2o_e').upper_bound = 1000
model.reactions.get_by_id('EX_n2_e').upper_bound = 1000

#2-step pathway
print('2-step pathway: NO3 -> NO2 -> N2O')
model.reactions.get_by_id('EX_no2_e').upper_bound = 0
model.reactions.get_by_id('EX_n2_e').upper_bound = 0

sol = model.optimize()
print(model.summary())

model.reactions.get_by_id('EX_no2_e').upper_bound = 1000
model.reactions.get_by_id('EX_n2_e').upper_bound = 1000

#3-step pathway
print('3-step pathway: NO3 -> NO2 -> N2O -> N2')
model.reactions.get_by_id('EX_no2_e').upper_bound = 0
model.reactions.get_by_id('EX_n2o_e').upper_bound = 0

sol = model.optimize()
print(model.summary())

model.reactions.get_by_id('EX_no2_e').upper_bound = 1000
model.reactions.get_by_id('EX_n2o_e').upper_bound = 1000


1-step pathway: NO3 -> NO2
Objective
1.0 Growth = 0.3936086137748

Uptake
------
Metabolite     Reaction      Flux  C-Number C-Flux
     ca2_e     EX_ca2_e  0.001984         0  0.00%
      cl_e      EX_cl_e  0.001984         0  0.00%
 cobalt2_e EX_cobalt2_e 3.812E-05         0  0.00%
     cu2_e     EX_cu2_e 0.0002703         0  0.00%
     fe3_e     EX_fe3_e  0.005536         0  0.00%
       h_e       EX_h_e     7.336         0  0.00%
       k_e       EX_k_e    0.0744         0  0.00%
  lac__D_e  EX_lac__D_e        10         0  0.00%
     mg2_e     EX_mg2_e  0.003307         0  0.00%
     mn2_e     EX_mn2_e 0.0002634         0  0.00%
     nac_e     EX_nac_e 0.0008683         0  0.00%
     nh4_e     EX_nh4_e     4.267         0  0.00%
     no3_e     EX_no3_e     26.47         0  0.00%
      pi_e      EX_pi_e    0.3759         0  0.00%
     so4_e     EX_so4_e   0.09545         0  0.00%
     thm_e     EX_thm_e   8.5E-05         0  0.00%
     zn2_e     EX_zn2_e   0.00013         0  0.00%



Select carbon sources

In [7]:
carbon_source = []

for ex in model.exchanges:
    if ex.id in media_models:
        ex.lower_bound = -1000
    else:
        ex.lower_bound = 0

#without o2 and with no3 
model.reactions.get_by_id('EX_o2_e').lower_bound = 0
model.reactions.get_by_id('EX_no3_e').lower_bound = -1000

#carbon source
for ex in model.exchanges:
    if not ex.id in media_models:
        ex.lower_bound = -10
        sol = model.optimize()
        if sol.objective_value>0.00001:
            print(ex.id, sol.objective_value)
            carbon_source.append(ex.id)

        ex.lower_bound = 0

Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.


EX_12ppd__S_e 0.49201076721849996
EX_glc__D_e 0.9152650001314078
EX_3amp_e 0.8077826055226288
EX_ala__L_e 0.39360861377480116
EX_4abut_e 0.6150134590231268
EX_acald_e 0.3007659749565217
EX_ac_e 0.18508675381939793
EX_actn__R_e 0.5552602614581953
EX_ad_e 0.19665467593311145
EX_mal__L_e 0.40590888295526356
EX_alaala_e 0.8118177659105317
EX_cys__L_e 0.3601784229798998
EX_gln__L_e 0.5959143016330529
EX_gly_e 0.17766513746838397
EX_ser__L_e 0.31979724744308996
EX_thr__L_e 0.5105592706552661
EX_arg__L_e 0.7562203522959206
EX_cgly_e 0.5982795930921831
EX_malt_e 1.7796819446999441
EX_malttr_e 2.6949469448313566
EX_orn_e 0.679863314231846
EX_asp__L_e 0.44433079255891206
EX_pro__L_e 0.7149216183756896
EX_cellb_e 1.8305300002628095
EX_meoh_e 0.13767735293742642
EX_cit_e 0.6029223497265987
EX_succ_e 0.43050942131618686
EX_glu__L_e 0.6273137282035877
EX_dca_e 1.4344223421003595
EX_ddca_e 1.8508675381939848
EX_dha_e 0.4428096904966488
EX_lac__D_e 0.39360861377480016
EX_hdca_e 2.498671176561887
EX_hd

In [8]:
print('Number of carbon sources: ', len(carbon_source))

Number of carbon sources:  63


In [9]:
#load coli's model to get carbon atoms from metabs
coli = read_sbml_model('/Users/magdalena/polybox/research/projecto_Sebastian/iJO1366.xml')

Get yield in the different carbon sources

In [10]:
d_yield = {}

for cs in carbon_source:
    
    #get carbon atoms in metab
    metab_id = cs.split('EX_')[1]

    d_yield[metab_id] = {}
    
    if metab_id in [m.id for m in coli.metabolites]:
        metab_Cnumber = coli.metabolites.get_by_id(metab_id).elements['C']
        metab_name = coli.metabolites.get_by_id(metab_id).name
        d_yield[metab_id]['name'] = metab_name
        d_yield[metab_id]['yield'] = []
        
        #set medium
        for ex in model.exchanges:
            if ex.id in media_models:
                ex.lower_bound = -1000
            else:
                ex.lower_bound = 0


        #carbon source
        model.reactions.get_by_id(cs).lower_bound = -10

        #without o2 and with no3 
        model.reactions.get_by_id('EX_o2_e').lower_bound = 0
        model.reactions.get_by_id('EX_no3_e').lower_bound = -1000

        #without no secretion (as in Xin's model)
        model.reactions.get_by_id('EX_no_e').upper_bound = 0

        ##
        #change the N product secreted - 1, 2 or 3 steps

        #1-step pathway
        #print('1-step pathway: NO3 -> NO2')
        model.reactions.get_by_id('EX_n2o_e').upper_bound = 0
        model.reactions.get_by_id('EX_n2_e').upper_bound = 0

        sol = model.optimize()
        biomass_carbon = 10*metab_Cnumber - sol['EX_co2_e']
        carbon_yield = biomass_carbon/(10*metab_Cnumber)
        d_yield[metab_id]['yield'].append(carbon_yield)

        model.reactions.get_by_id('EX_n2o_e').upper_bound = 1000
        model.reactions.get_by_id('EX_n2_e').upper_bound = 1000

        #2-step pathway
        #print('2-step pathway: NO3 -> NO2 -> N2O')
        model.reactions.get_by_id('EX_no2_e').upper_bound = 0
        model.reactions.get_by_id('EX_n2_e').upper_bound = 0

        sol = model.optimize()
        biomass_carbon = 10*metab_Cnumber - sol['EX_co2_e']
        carbon_yield = biomass_carbon/(10*metab_Cnumber)
        d_yield[metab_id]['yield'].append(carbon_yield)

        model.reactions.get_by_id('EX_no2_e').upper_bound = 1000
        model.reactions.get_by_id('EX_n2_e').upper_bound = 1000

        #3-step pathway
        #print('3-step pathway: NO3 -> NO2 -> N2O -> N2')
        model.reactions.get_by_id('EX_no2_e').upper_bound = 0
        model.reactions.get_by_id('EX_n2o_e').upper_bound = 0

        sol = model.optimize()
        biomass_carbon = 10*metab_Cnumber - sol['EX_co2_e']
        carbon_yield = biomass_carbon/(10*metab_Cnumber)
        d_yield[metab_id]['yield'].append(carbon_yield)

        model.reactions.get_by_id('EX_no2_e').upper_bound = 1000
        model.reactions.get_by_id('EX_n2o_e').upper_bound = 1000

        ###
        #carbon source back to zero
        model.reactions.get_by_id(cs).lower_bound = 0

Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
Could not identify an external compartment by name and choosing one with the mos

Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
Could not identify an external compartment by name and choosing one with the mos

In [11]:
no_info = 0
for cs in d_yield:
    if 'yield' in d_yield[cs]: 
        if round(d_yield[cs]['yield'][0],3)<round(d_yield[cs]['yield'][1],3) or round(d_yield[cs]['yield'][1],3)<round(d_yield[cs]['yield'][2],3):
            print(cs, d_yield[cs])
    else:
        no_info += 1

In [12]:
print('Metabs not in coli: ', no_info)

Metabs not in coli:  4


Write table with yields

In [13]:
f = open('Denitrifier_yields_different_carbonsources.tsv','w')
f.write('Metab id \t Metab name \t Carbon yield NO3->NO2 \t Carbon yield NO3->NO2->N20 \t Carbon yield NO3->NO2->N20->N2')
for cs in d_yield:
    if 'yield' in d_yield[cs]: 
        f.write('\n')
        f.write(cs + '\t' + d_yield[cs]['name'] + '\t' + str(round(d_yield[cs]['yield'][0],3)) + '\t' + str(round(d_yield[cs]['yield'][1],3)) + '\t' + str(round(d_yield[cs]['yield'][2],3)))
        
f.close()        